In [2]:
import uuid
from azure.ai.ml import command
from azure.ai.ml.entities import Data
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    CodeConfiguration,
    Environment,
)

ml_client = MLClient.from_config(credential=DefaultAzureCredential())

Found the config file in: /mnt/batch/tasks/shared/LS_root/mounts/clusters/jcharley4/code/Users/jcharley/config.json


Batch endpoints are endpoints that are used to do batch inferencing on large volumes of data over a period of time. 

Batch endpoints receive pointers to data and run jobs asynchronously to process the data in parallel on compute clusters. Batch endpoints store outputs to a data store for further analysis.

### STEP 1 : create batch compute (cluster)

In [3]:
# create compute cluster to be used by batch cluster
from azure.ai.ml.entities import AmlCompute

my_cluster = AmlCompute(
    name="batch-cluster",
    type="amlcompute", 
    size="STANDARD_DS3_V2", 
    min_instances=0, 
    max_instances=2,
    location="westeurope", 	
)
ml_client.compute.begin_create_or_update(my_cluster)

AmlCompute({'type': 'amlcompute', 'created_on': None, 'provisioning_state': 'Succeeded', 'provisioning_errors': None, 'name': 'batch-cluster', 'description': None, 'tags': {}, 'properties': {}, 'id': '/subscriptions/66914bb5-9cb2-4f6d-a84d-8ff900446b22/resourceGroups/Learning/providers/Microsoft.MachineLearningServices/workspaces/test_learn/computes/batch-cluster', 'base_path': './', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7f4561252440>, 'resource_id': None, 'location': 'westeurope', 'size': 'STANDARD_DS3_V2', 'min_instances': 0, 'max_instances': 2, 'idle_time_before_scale_down': 120.0, 'identity': None, 'ssh_public_access_enabled': True, 'ssh_settings': None, 'network_settings': <azure.ai.ml.entities._compute.compute.NetworkSettings object at 0x7f4561253070>, 'tier': 'dedicated'})

### STEP2 : Create Batch Endpoint

In [4]:
# Creating a unique name for the endpoint
batch_endpoint_name = "taxi-batch-" + str(uuid.uuid4())[:8]
print(batch_endpoint_name)

taxi-batch-c67fd0de


In [5]:
# create batch endpoint
from azure.ai.ml.entities import BatchEndpoint

batch_endpoint = BatchEndpoint(
    name=batch_endpoint_name,
    description="Taxi batch endpoint",
    tags={"model": "taxi-model@latest"},
)

ml_client.begin_create_or_update(batch_endpoint)


BatchEndpoint({'scoring_uri': 'https://taxi-batch-c67fd0de.westeurope.inference.ml.azure.com/jobs', 'swagger_uri': None, 'provisioning_state': 'Succeeded', 'name': 'taxi-batch-c67fd0de', 'description': 'Taxi batch endpoint', 'tags': {'model': 'taxi-model@latest'}, 'properties': {'BatchEndpointCreationApiVersion': '2022-05-01', 'azureml.onlineendpointid': '/subscriptions/66914bb5-9cb2-4f6d-a84d-8ff900446b22/resourcegroups/learning/providers/microsoft.machinelearningservices/workspaces/test_learn/onlineendpoints/taxi-batch-c67fd0de', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/66914bb5-9cb2-4f6d-a84d-8ff900446b22/providers/Microsoft.MachineLearningServices/locations/westeurope/mfeOperationsStatus/be:75310cf9-c772-4f21-9638-4129bab98f70:185b71c4-b068-4977-b099-3e94f2e8d846?api-version=2022-05-01'}, 'id': '/subscriptions/66914bb5-9cb2-4f6d-a84d-8ff900446b22/resourceGroups/Learning/providers/Microsoft.MachineLearningServices/workspaces/test_learn/batchEndpoints/tax

### STEP3 : Create Batch Deployment

In [8]:
# create batch deployment
from azure.ai.ml.entities import BatchDeployment, Model, Environment
from azure.ai.ml.constants import BatchDeploymentOutputAction

model = "taxi-model@latest"

batch_deployment = BatchDeployment(
    name="taxi-batch-dp",
    description="this is a sample batch deployment",
    endpoint_name=batch_endpoint_name,
    model=model,
    compute="batch-cluster",
    instance_count=2,
    max_concurrency_per_instance=2,
    mini_batch_size=10,
    output_action=BatchDeploymentOutputAction.APPEND_ROW,
    output_file_name="predictions.csv",
)

ml_client.begin_create_or_update(batch_deployment)


BatchDeployment({'endpoint_name': 'taxi-batch-c67fd0de', 'type': None, 'name': 'taxi-batch-dp', 'description': 'this is a sample batch deployment', 'tags': {}, 'properties': {}, 'id': '/subscriptions/66914bb5-9cb2-4f6d-a84d-8ff900446b22/resourceGroups/Learning/providers/Microsoft.MachineLearningServices/workspaces/test_learn/batchEndpoints/taxi-batch-c67fd0de/deployments/taxi-batch-dp', 'base_path': './', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7f4553479f60>, 'model': '/subscriptions/66914bb5-9cb2-4f6d-a84d-8ff900446b22/resourceGroups/Learning/providers/Microsoft.MachineLearningServices/workspaces/test_learn/models/taxi-model/versions/2', 'code_configuration': None, 'environment': None, 'environment_variables': {}, 'compute': '/subscriptions/66914bb5-9cb2-4f6d-a84d-8ff900446b22/resourceGroups/Learning/providers/Microsoft.MachineLearningServices/workspaces/test_learn/computes/batch-cluster', 'resources': {'instance_count': 2, 'properties': {}}

In [9]:
batch_endpoint = ml_client.batch_endpoints.get(batch_endpoint_name)
batch_endpoint.defaults.deployment_name = batch_deployment.name
ml_client.batch_endpoints.begin_create_or_update(batch_endpoint)

BatchEndpoint({'scoring_uri': 'https://taxi-batch-c67fd0de.westeurope.inference.ml.azure.com/jobs', 'swagger_uri': None, 'provisioning_state': 'Succeeded', 'name': 'taxi-batch-c67fd0de', 'description': 'Taxi batch endpoint', 'tags': {'model': 'taxi-model@latest'}, 'properties': {'BatchEndpointCreationApiVersion': '2022-05-01', 'azureml.onlineendpointid': '/subscriptions/66914bb5-9cb2-4f6d-a84d-8ff900446b22/resourcegroups/learning/providers/microsoft.machinelearningservices/workspaces/test_learn/onlineendpoints/taxi-batch-c67fd0de', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/66914bb5-9cb2-4f6d-a84d-8ff900446b22/providers/Microsoft.MachineLearningServices/locations/westeurope/mfeOperationsStatus/be:75310cf9-c772-4f21-9638-4129bab98f70:48008786-1527-4c07-b548-ae83305b45ae?api-version=2022-05-01'}, 'id': '/subscriptions/66914bb5-9cb2-4f6d-a84d-8ff900446b22/resourceGroups/Learning/providers/Microsoft.MachineLearningServices/workspaces/test_learn/batchEndpoints/tax

### STEP 4 : CONSUME

In [12]:
# invoke and test endpoint
# Invoke by python sdk or CLI
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes, InputOutputModes

input = Input(path="./taxi-batch.csv", 
              type=AssetTypes.URI_FILE, 
              mode=InputOutputModes.DOWNLOAD)


# invoke the endpoint for batch scoring job
ml_client.batch_endpoints.invoke(
    endpoint_name=batch_endpoint_name,
    input=input,
    deployment_name=batch_deployment.name
)


Uploading taxi-batch.csv (< 1 MB): 100%|██████████| 133k/133k [00:00<00:00, 7.88MB/s]


